In [ ]:
import sys
import os
import torch as T
import numpy as np
import random

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [2]:
def set_global_seed(seed: int):
    T.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    if T.cuda.is_available():
        T.cuda.manual_seed_all(seed)

In [3]:
SEED = 42

device = T.device('cuda' if T.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

set_global_seed(SEED)

Using device: cuda


## Dataset Loading

In [4]:
from torch.utils.data import DataLoader
from datasets.custom_dataset import CustomWCSTDataset

### 1. Dataset Hyperparameters

In [5]:
BATCH_SIZE = 64
TOTAL_BATCHES = 750
TRAIN_TEST_SPLIT_RATIO = 0.6
VALIDATION_TEST_SPLIT_RATIO = 0.5

### 2. Loading Dataset

In [ ]:
train_size = int(TOTAL_BATCHES * TRAIN_TEST_SPLIT_RATIO)
validation_size = int((TOTAL_BATCHES - train_size) * VALIDATION_TEST_SPLIT_RATIO)
test_size = TOTAL_BATCHES - train_size - validation_size

train_datasets = {
    "color": CustomWCSTDataset(total_batches=train_size // 3, fixed_context=0, sample_batch_size=BATCH_SIZE, allow_switch=False),
    "shape": CustomWCSTDataset(total_batches=train_size // 3, fixed_context=1, sample_batch_size=BATCH_SIZE, allow_switch=False),
    "quantity": CustomWCSTDataset(total_batches=train_size // 3, fixed_context=2, sample_batch_size=BATCH_SIZE, allow_switch=False)
}

train_loaders = {
    ctx: DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True)
    for ctx, ds in train_datasets.items()
}

validation_datasets = {
    "color": CustomWCSTDataset(total_batches=validation_size // 3, fixed_context=0, sample_batch_size=BATCH_SIZE, allow_switch=False),
    "shape": CustomWCSTDataset(total_batches=validation_size // 3, fixed_context=1, sample_batch_size=BATCH_SIZE, allow_switch=False),
    "quantity": CustomWCSTDataset(total_batches=validation_size // 3, fixed_context=2, sample_batch_size=BATCH_SIZE, allow_switch=False)
}

validation_loaders = {
    ctx: DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True)
    for ctx, ds in validation_datasets.items()
}

test_dataset = CustomWCSTDataset(
        total_batches=test_size, sample_batch_size=BATCH_SIZE
    )
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

[Dataset Init] Fixed context: 0
[Dataset Init] Fixed context: 1
[Dataset Init] Fixed context: 2
[Dataset Init] Context switched -> 2
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 2
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 2
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 2
[Dataset Init] Context switched -> 1
[Dataset Init] Context switched -> 2
[Dataset Init] Context switched -> 0
[Dataset Init] Context switched -> 2
[Dataset Init] C

## Transformer Model Creation & Training

In [7]:
import itertools
import numpy as np
from collections import defaultdict
from torch.utils.data import DataLoader
from torch import nn, optim

from src.transformer import BaselineTransformer

### 1. Train, Validate and Test Model Functions

In [8]:
def validate_per_context(
        model: nn.Module, criterion: nn.CrossEntropyLoss, loaders: dict[str, DataLoader], device: T.device | str ="cpu"
    ):
    model.eval()
    results = {}

    with T.no_grad():
        for ctx_name, loader in loaders.items():
            total_loss, total_correct, total_samples = 0.0, 0, 0
            for input_seq, target in loader:
                input_seq, target = input_seq.to(device), target.view(-1).to(device)

                logits = model(input_seq)
                logits = logits[:, -1, :]
                
                loss = criterion(logits, target)

                preds = logits.argmax(dim=1)
                total_loss += loss.item() * target.size(0)
                total_correct += (preds == target).sum().item()
                total_samples += target.size(0)

            avg_loss = total_loss / total_samples
            avg_acc = total_correct / total_samples

            results[ctx_name] = {
                "loss": avg_loss,
                "acc": avg_acc,
                "perplexity": np.exp(avg_loss)
            }
    return results

In [10]:
def train_model_round_robin(
    train_loaders: dict[str, DataLoader], validation_loaders: dict[str, DataLoader], model: nn.Module,
    criterion: nn.CrossEntropyLoss, optimizer: optim.Optimizer, max_epochs: int = 20,
    device: str | T.device = "cpu"
):

    history = defaultdict(list)

    for epoch in range(max_epochs):
        print(f"\n[Epoch {epoch + 1}/{max_epochs}]")
        print("-" * 60)

        model.train()
        epoch_losses, total_correct, total_samples = [], 0, 0

        # --- Build a round-robin iterator across all loaders ---
        loaders_cycle = itertools.cycle(train_loaders.items())
        active_iters = {ctx: iter(dl) for ctx, dl in train_loaders.items()}

        # Find smallest loader length to roughly balance epoch size
        min_len = min(len(dl) for dl in train_loaders.values())
        total_batches = min_len * len(train_loaders)

        for batch_idx in range(total_batches):
            context, _ = next(loaders_cycle)
            loader_iter = active_iters[context]

            try:
                input_seq, target = next(loader_iter)
            except StopIteration:
                # Restart exhausted iterator
                active_iters[context] = iter(train_loaders[context])
                input_seq, target = next(active_iters[context])

            input_seq, target = input_seq.to(device), target.view(-1).to(device)

            # Forward pass
            logits = model(input_seq)  # [batch, seq_len, vocab]
            logits = logits[:, -1, :]  # predict final token only
            loss = criterion(logits, target)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            preds = logits.argmax(dim=1)
            total_correct += (preds == target).sum().item()
            total_samples += target.size(0)
            epoch_losses.append(loss.item())

            if batch_idx % 50 == 0 or batch_idx == total_batches - 1:
                print(f"[{context}] Batch {batch_idx+1}/{total_batches} | Loss: {loss.item():.4f}")

        # --- Epoch stats ---
        train_loss = np.mean(epoch_losses)
        train_acc = total_correct / total_samples
        train_perplexity = np.exp(train_loss)

        history["train_losses"].append(train_loss)
        history["train_accs"].append(train_acc)
        history["train_perplexities"].append(train_perplexity)

        print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train Perplexity: {train_perplexity:.4f}")

        # --- Validation ---
        context_results = validate_per_context(model, criterion, validation_loaders, device)
        val_loss = []
        val_acc = []
        val_perplexity = []
        for ctx, metrics in context_results.items():
            print(f"[Validation] {ctx}: Loss: {metrics['loss']:.4f} | Acc: {metrics['acc']:.4f} | Perplexity: {metrics['perplexity']:.4f}")
            val_loss.append(metrics['loss'])
            val_acc.append(metrics['acc'])
            val_perplexity.append(metrics['perplexity'])

        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)
        val_perplexity = np.mean(val_perplexity)

        print(f"[Validation Summary] Val Loss: {np.mean(val_loss):.4f} | "
              f"Val Acc: {np.mean(val_acc):.4f} | Val Perplexity: {np.mean(val_perplexity):.4f}", end="\n\n")
        print("-" * 40)

        history["val_context"].append(context_results)
        history["val_losses"].append(val_loss)
        history["val_accs"].append(val_acc)
        history["val_perplexities"].append(val_perplexity)

    print("\nTraining complete")

    return history


In [11]:
def test_model(test_loader: DataLoader, model: nn.Module, criterion: nn.CrossEntropyLoss, device: str | T.device = "cpu"):

    model.eval()
    test_batch_losses = []
    test_correct = 0
    test_tokens = 0

    with T.no_grad():
        for input_seq, target in test_loader:
            input_seq, target = input_seq.to(device), target.view(-1).to(device)

            # Forward pass
            logits = model(input_seq)  # [batch, seq_len, vocab]
            logits = logits[:, -1, :]
            loss = criterion(logits, target)

            preds = logits.argmax(dim=1)
            test_correct += (preds == target).sum().item()
            test_tokens += target.size(0)

            test_batch_losses.append(loss.item())

    test_loss = np.mean(test_batch_losses)
    test_acc = test_correct / test_tokens
    test_perplexity = np.exp(test_loss)

    return {
        "test_loss": test_loss,
        "test_acc": test_acc,
        "test_perplexity": test_perplexity
    }

### 2. Create & Train Transformer Model

In [12]:
VOCABULARY_SIZE = 70        # 64 cards + 4 categories + SEP + EOS
EMBEDDING_SIZE = 256        # larger embedding to capture card features
N_ATTENTION_HEADS = 8       # more heads for better multi-feature attention
N_BLOCKS = 6                # same depth as before
MAX_SEQUENCE_LENGTH = 10    # longer max sequence to accommodate multiple past trials
FF_DIMS = 1024               # larger feedforward layer for better representation
DROPOUT_PROB = 0.2        # reduce dropout slightly to retain signal in small batches

LEARNING_RATE = 3e-4
BATCH_SIZE = 64
WEIGHT_DECAY = 1e-2
WARMUP_STEPS = 400
LABEL_SMOOTHING = 0.1
MAX_EPOCHS = 100
SEEDS = [31, 42, 45, 69, 420]

In [13]:
results = []

for seed in SEEDS:
    print(f"\n--- Training with Seed {seed} ---")
    set_global_seed(seed)
    
    transformer = BaselineTransformer(
        VOCABULARY_SIZE, EMBEDDING_SIZE, N_ATTENTION_HEADS,
        N_BLOCKS, MAX_SEQUENCE_LENGTH, FF_DIMS, DROPOUT_PROB, device=device
    )

    criterion =  nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
    optimizer = optim.AdamW(transformer.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=WARMUP_STEPS)  

    result = train_model_round_robin(
        train_loaders, validation_loaders, transformer, criterion,
        optimizer, max_epochs=MAX_EPOCHS, device=device
    )

    test_results = test_model(test_loader, transformer, criterion, device)
    print(f"[Test] Loss: {test_results["test_loss"]:.4f} | Acc: {test_results["test_acc"]:.4f} | Perplexity: {test_results["test_perplexity"]:.4f}")

    results.append({**result, **test_results})


--- Training with Seed 31 ---

[Epoch 1/100]
------------------------------------------------------------
[color] Batch 1/450 | Loss: 181.2520
[quantity] Batch 51/450 | Loss: 6.3013
[shape] Batch 101/450 | Loss: 4.8506
[color] Batch 151/450 | Loss: 3.7360
[quantity] Batch 201/450 | Loss: 3.3604
[shape] Batch 251/450 | Loss: 3.4322
[color] Batch 301/450 | Loss: 3.1310
[quantity] Batch 351/450 | Loss: 3.0025
[shape] Batch 401/450 | Loss: 3.0368
[quantity] Batch 450/450 | Loss: 2.8282
[Epoch 1] Train Loss: 4.7837 | Train Acc: 0.2386 | Train Perplexity: 119.5485
[Validation] color: Loss: 2.3372 | Acc: 0.2532 | Perplexity: 10.3527
[Validation] shape: Loss: 2.3346 | Acc: 0.2543 | Perplexity: 10.3256
[Validation] quantity: Loss: 2.3365 | Acc: 0.2521 | Perplexity: 10.3454
[Validation Summary] Val Loss: 2.3361 | Val Acc: 0.2532 | Val Perplexity: 10.3412

----------------------------------------

[Epoch 2/100]
------------------------------------------------------------
[color] Batch 1/450 | Lo

### 3. Save Training & Test Results 

In [16]:
import json

In [17]:
for i, result in enumerate(results):
    with open(f"./../results/baseline_transformer_round_robin_performace_seed-{SEEDS[i]}.json", "w") as file:
        json.dump(result, file, indent=4)